# Introdução

O objetivo desse notebook é analisar alguns dados sobre os clientes de um shopping e depois aplicar um algoritmo de **aprendizado não-supervisionado** com intuito de agrupar os clientes em "clusters". Pela natureza desse tipo de aprendizado, note que não haverá uma coluna alvo (target), e sim a utilização de todas as "features" para realizar o agrupamento.

# Bibliotecas necessárias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

# Análise dos dados

In [ ]:
df = pd.read_csv('../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
df.head()

Entenda as colunas como:
- **CustomerID**: identificador único de cada cliente
- **Gender**: gênero do cliente
- **Age**: idade do cliente
- **Annual Income (k$**): renda anual do cliente
- **Spending Score (1-100)**: pontuação atribuída pelo shopping com base no comportamento do cliente e natureza dos gastos.

#### Shape

In [ ]:
print('Linhas: ', df.shape[0])
print('Colunas: ', df.shape[1])

É um conjunto de dados relativamente pequeno, não só em questão de linhas, mas também com poucas "features".

#### Dados nulos

In [ ]:
df.isnull().sum()

Como já esperado de um conjunto de dados tão pequeno, não há dados nulos.

#### Info

In [ ]:
df.info()

O pandas classificou corretamente o tipo dos dados, porém, caso fosse feita alguma aplicação de um modelo de **aprendizado supervisionado**, seria interessante transformar as variáveis (menos "Annual Income (k$)") em categóricas, e logo após aplicar o "Onehot Encoder".

### Distribuição de algumas "features"

In [ ]:
plt.figure(figsize=(16, 5))

plt.subplot(1, 3, 1)
sns.distplot(df['Annual Income (k$)'], color='g')
plt.title('Renda anual')

plt.subplot(1, 3, 2)
sns.distplot(df['Age'])
plt.title('Idade')

plt.subplot(1, 3, 3)
sns.distplot(df['Spending Score (1-100)'], color='r')
plt.title('Pontuação')

plt.show()

#### Renda anual

É possível notar que quase não há clientes que ganham menos de **20k de dólares anualmente**, enquanto há uma quantidade baixa de clientes que ganham mais que **110k de doláres**. A maioria desses clientes ganha entre **50k e ~62k, e entre 75k e ~85k**.

#### Idade

Há picos entre **20 anos e 50 anos**, porém acima disso a quantidade de clientes é bem baixa.

#### Pontuação

Assemelha-se à uma distribuição normal. A maior parte dos clientes possuem pontuações ~45 e ~55.

### Quantidade de clientes por gênero

In [ ]:
plt.figure(figsize=(5, 5))

plt.pie(df.Gender.value_counts(), colors=['pink', 'lightblue'], explode=[0, 0.1], labels=['Feminino', 'Masculino'], autopct='%.2f%%')
plt.title('Gênero', fontsize=14)
plt.show()

Há uma diferença de 12% entr os gêneros, onde o público feminino representa a maioria dos clientes. 

### Correlações

In [ ]:
plt.figure(figsize=(10, 6))

sns.heatmap(df.corr(), vmin=-1, vmax=1, cmap='Blues', annot=True, linewidths=2, linecolor='black')

plt.show()

Apesar desse mapa de calor não nos dar muitas informações úteis, é interessante notar essa correlação negativa entre a **idade** e a **pontuação**, ou seja, quanto mais velho o cliente, menor sua pontuação, provavelmente pelo fato de fazer menos compras.

### Gênero vs Pontuação

In [ ]:
sns.boxplot(df['Gender'], df['Spending Score (1-100)'], palette=['lightblue', 'pink'])
plt.title('Gender vs Spending Score', fontsize = 20)
plt.show()

Não há muita diferença nas pontuações quando se trata do gênero, mas é possível ver que as mulheres normalmente estão melhores pontuadas.

### Distribuição de renda anual vs Pontuação

In [ ]:
plt.figure(figsize=(15, 4))

plt.subplot(1, 2, 1)
male_income = df.loc[df.Gender == 'Male', 'Annual Income (k$)']
male_score = df.loc[df.Gender == 'Male', 'Spending Score (1-100)']
sns.kdeplot(male_income, color='b', shade=True)
sns.kdeplot(male_score, color='g', shade=True)
plt.title('Homem')

plt.subplot(1, 2, 2)
female_income = df.loc[df.Gender == 'Female', 'Annual Income (k$)']
female_score = df.loc[df.Gender == 'Female', 'Spending Score (1-100)']
sns.kdeplot(female_income, color='pink', shade=True)
sns.kdeplot(female_score, color='g', shade=True)
plt.title('Mulher')

plt.legend()
plt.tight_layout()
plt.show()

Mais homens possuem um ganho anual maior, porém suas pontuações são um pouco mais baixas que as das mulheres, que possuem uma renda anual menor.

# Modelagem

Durante a criação de nosso modelo, utilizaremos o **KMeans**, além do **Elbow method** para mensurar o melhor número de clusters. Para saber mais sobre isso, consulte:
- [Kmeans](https://www.geeksforgeeks.org/k-means-clustering-introduction)
- [Elbow Method](https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans)
- [Evaluating Clusters](https://medium.com/@ODSC/unsupervised-learning-evaluating-clusters-bd47eed175ce)

Iremos utilizar as colunas **Gender, Age e Annual Income (k$)** para efeturar o treinamento e verificar o número de clusters.

In [ ]:
x = df.iloc[:, [2, 4]].values

#### Elbow Method

In [ ]:
plt.figure(figsize=(8, 4))
# Within cluster sum o errors
# Inicializa uma lista varia
wcss = []

# Itera entre 1 e 10 clusters
for i in range(1, 11):
    # Cria uma instância do KMeans
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=500, n_init=10, random_state=42)
    # Treina o modelo
    kmeans.fit(x)
    # Armazena a inertia na lista criada anteriormente
    wcss.append(kmeans.inertia_)
    
plt.plot(range(1, 11), wcss)
plt.xlabel('Quantidade de clusters')
plt.grid(True)
plt.show()

Como podemos ver, após 4 **clusters**, a inercia decresce bem pouco, ou seja, não há uma mudança significante no seu valor.

#### Treinamento final

In [ ]:
# modelo com 4 clusters
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=500, n_init=10, random_state=42)
preds = kmeans.fit_predict(x)

In [ ]:
plt.figure(figsize=(12, 8))

# Aqui iremos visualizar os resultados dos 4 clusters
plt.scatter(x[preds==0, 0], x[preds==0, 1], s=100, c='red', alpha=0.5, label='Comum')
plt.scatter(x[preds==1, 0], x[preds==1, 1], s=100, c='green', alpha=0.5, label='Ideal')
plt.scatter(x[preds==2, 0], x[preds==2, 1], s=100, c='blue', alpha=0.5, label='Usual')
plt.scatter(x[preds==3, 0], x[preds==3, 1], s=100, c='cyan', alpha=0.5, label='Usual - Mais velho')

# Centróides
plt.title('Clusters da Idade', fontsize=14)
plt.xlabel('Idade')
plt.ylabel('Pontuação')

plt.legend()
plt.show()

# Conclusão

De acordo com o gráfico, podemos observar 4 clusters distintos os quais classifiquei como:
- Cluster 1 (verde): são os clientes **ideais**. São jovens, e com pontuações mais altas, fato provavelmente causado pela disposição em gastar dinheiro
- Cluster 2 (vermelho): são os clientes **comuns**, ou seja, com idades diversas e pontuações mais baixas. São aqueles que provavelmente fizeram poucas compras
- Cluster 3 (azul): são os clientes **usuais**. São jovens e jovens adultos, e possuem pontuações medianas
- Cluster 4 (ciano): também são clientes **usuais**, porém dessa vez com mais idade.

A recomendação é ter um olhar mais atento ao grupo **ideal**, traçando planos e estratégias para fidelização.